In [1]:
# Importing all libraries required
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from keras.utils.np_utils import to_categorical
import re
import json
import nltk
from pprint import pprint
import nltk, re, pprint, sys
from nltk.tokenize import *
from nltk import  word_tokenize, pos_tag, wordpunct_tokenize
from nltk.collocations import *
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import word_tokenize, pos_tag, ne_chunk
import json
from collections import defaultdict,Counter
from pprint import pprint
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import gensim
import pickle
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.neural_network import MLPClassifier
import sklearn as sk
from itertools import product
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM,Embedding

Using TensorFlow backend.


In [2]:
#Importing the data
with open('dev-v1.1.json') as data_file:    
    data = json.load(data_file)

In [3]:
#Number and type of questions in the dataset
num_questions=0
num_paras=0
question_types_count={'what':0, 'why':0, 'when':0,'where':0, 'how':0, 'which':0,'who':0, 'whom':0}
for i in range(len(data['data'])):
    paragraphs  = data['data'][i]['paragraphs']
    qs =[]
    for p in paragraphs:  
        questions = p['qas']
        for q in questions:
            qs.append(q['question'])
    num_questions += len (qs)
    num_paras += len(paragraphs)
    question_words = ['what', 'why', 'when','where', 'how', 'which','who', 'whom']
    all_qs = " ".join(q.lower() for q in qs)
    word_count = Counter(all_qs.split())
    for q in question_words:
        question_types_count[q]+=word_count[q]
print ('Total number of paragraphs in train set : ',num_paras)      
print ('Number of question answer pairs in train set : ',num_questions)
for ques,count in question_types_count.items():
    print("Question type:",ques,"\tCount:",count  )

Total number of paragraphs in train set :  2067
Number of question answer pairs in train set :  10570
Question type: what 	Count: 5895
Question type: why 	Count: 158
Question type: when 	Count: 857
Question type: where 	Count: 499
Question type: how 	Count: 1241
Question type: which 	Count: 748
Question type: who 	Count: 1280
Question type: whom 	Count: 23


In [4]:
#Data processing functions
def iter_ngrams(doc, n):
    """Return a generator over ngrams of a document.
    Params:
      doc...list of tokens
      n.....size of ngrams"""
    return (doc[i: i + n] for i in range(len(doc) - n + 1))

def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    #print(sentences)
    words = [re.findall('\w+', sent.lower()) for sent in sentences]
    tags = [nltk.pos_tag(word) for word in words]
    s = []
    t = []
    for tag in tags:
        s.append([a[0].lower() for a in tag])
        t.append([a[1] for a in tag])
    return s,t
def get_NER(document):
    sentences = nltk.sent_tokenize(document)
    s = []
    t = []
    n = []
    for sentence in sentences: 
        ne_tree = ne_chunk(pos_tag(word_tokenize(sentence)))
        iob_tagged = tree2conlltags(ne_tree)
        s.append([each[0] for each in iob_tagged])
        t.append([each[1] for each in iob_tagged])
        n.append([each[2] for each in iob_tagged])
    return s,t,n,sentence

In [5]:
#Rule based question answer 
questotag={"when":'CD',
           "who":['B-PERSON','I-PERSON'],
           "whom":['B-PERSON','I-PERSON'],
           "where":['B-GPE','I-GPE','B-ORGANIZATION','I-ORGANIZATION'],
           "what":['B-PERSON','I-PERSON','B-ORGANIZATION','I-ORGANIZATION'],
           "what place":['B-GPE','I-GPE','B-ORGANIZATION','I-ORGANIZATION'],
           "how many":"CD"}

In [6]:
# functions to process paragraphs and sentences
def get_paras(topic, category):
    para_details=[]
    paragraphs  = topic['paragraphs'] 
    for p in paragraphs:
        cat_qs=[]
        cat_ans=[]
        text = p['context']
        questions = p['qas']
        for q in questions:
            if category in (q['question']).lower()[:5]:
                cat_qs.append(q['question'])
                cat_ans.append(q['answers'][0]['text'])
        
        if (len(cat_qs)>0):
            para_details.append((text,cat_qs,cat_ans))
    return para_details

def get_ans_sentences(context,question):
    ques = re.findall('\w+', question[:-1].lower())
    q = [tuple(ngram) for ngram in iter_ngrams(ques, 1)]
    
    n_grams =[]
    sentences = nltk.sent_tokenize(context)
    for d in sentences:
        d = re.findall('\w+', d.lower())
        n_grams.append([tuple(ng) for ng in iter_ngrams(d, 1)])
    possible_sentence=[]    
    
    for indx,t in enumerate(n_grams):
        if(len(set.intersection(set(q),set(t))))>=2:
            possible_sentence.append(indx)

    return possible_sentence

def print_possible_answers(para, sent_indices, category):
    atype = questotag[category]
    sentences,tags,ner,sentece = get_NER(para)
    predicted_answers =[]
    if len(sent_indices)>0:
        for each in (sent_indices):
            if(category in ['when','how']):
                for t in zip(tags[each],sentences[each]):
                    if(t[0] in atype):
                        predicted_answers.append(t[1])
            else:
                for t in zip(ner[each],sentences[each]):
                    if(t[0] in atype):
                        predicted_answers.append(t[1])
    return predicted_answers

def print_possible_sentence(para, sent_indices, category):
    atype = questotag[category]
    sentences,tags,ner,sentece = get_NER(para)
    predicted_sentences =[]
    if len(sent_indices)>0:
        for each in (sent_indices):
            if(category in ['when','how']):
                for t in zip(tags[each],sentences[each]):
                    #print(sentences[each])
                    if(t[0] in atype):
                        predicted_sentences.append(" ".join(sentences[each]))
            else:
                for t in zip(ner[each],sentences[each]):
                    #print(sentences[each])
                    if(t[0] in atype):
                        predicted_sentences.append(" ".join(sentences[each]))
        return predicted_sentences 

In [7]:
def process_train_data_possible_answers():
    category = questotag.keys()
    data2 = {}
    total1 = 0
    total2 = 0
    topics = data['data']
    for t in topics: 
        true_answer_cnt=0
        total_cnt=0
        for cat in category:
            data2[cat] = get_paras(t, cat)
        for cat in category:
            for i in range(0,len(data2[cat])):
                para = data2[cat][i][0]
                question = data2[cat][i][1][0]
                sent_indices = get_ans_sentences(para, question)
                sent_indices
                predicted_answers = print_possible_answers(para,sent_indices, cat)
                answer =  data2[cat][i][2][0]
                total_cnt+=1
                total1+=1
                for each in answer.split():
                    if(each in predicted_answers):
                        true_answer_cnt+=1
                        total2+=1
                        break
        if(total_cnt>0):
            print("number of answers matched with possible answers",true_answer_cnt,"from total of",total_cnt)
            print("Accuracy: ",true_answer_cnt/total_cnt)
    print("number of answers matched with possible answers",total2,"from total of",total1)
    print("Accuracy: ",total2/total1)

In [8]:
process_train_data_possible_answers()

number of answers matched with possible answers 72 from total of 117
Accuracy:  0.6153846153846154
number of answers matched with possible answers 49 from total of 100
Accuracy:  0.49
number of answers matched with possible answers 46 from total of 77
Accuracy:  0.5974025974025974
number of answers matched with possible answers 87 from total of 190
Accuracy:  0.45789473684210524
number of answers matched with possible answers 9 from total of 51
Accuracy:  0.17647058823529413
number of answers matched with possible answers 14 from total of 92
Accuracy:  0.15217391304347827
number of answers matched with possible answers 113 from total of 207
Accuracy:  0.5458937198067633
number of answers matched with possible answers 16 from total of 37
Accuracy:  0.43243243243243246
number of answers matched with possible answers 22 from total of 41
Accuracy:  0.5365853658536586
number of answers matched with possible answers 17 from total of 43
Accuracy:  0.3953488372093023
number of answers matched 

In [9]:
def process_train_data():
    category = questotag.keys()
    data2 = {}
    total1 = 0
    total2 = 0
    topics = data['data']
    with open("Question_answer","wb") as f:
        for t in topics: 
            true_answer_cnt=0
            total_cnt=0
            for cat in category:
                data2[cat] = get_paras(t, cat)
            for cat in category:
                for i in range(0,len(data2[cat])):
                    para = data2[cat][i][0]
                    question = data2[cat][i][1][0]
                    sent_indices = get_ans_sentences(para, question)
                    sent_indices
                    predicted_sentence = print_possible_sentence(para,sent_indices, cat)
                    answer =  data2[cat][i][2][0]
                    total_cnt+=1
                    total1+=1
                    cnd=0
                    if(predicted_sentence!=None):
                        for i in predicted_sentence:
                            if answer in i:
                                true_answer_cnt+=1
                                total2+=1
                                pickle.dump((question+" "+i,1),f)
                                cnd=1
                                break
                            else:
                                pickle.dump((question+" "+i,0),f)  
                    if(cnd==0):
                        pickle.dump((question+" "+answer,1),f)
    print("number of answers matched with possible answers",total2,"from total of",total1)
    print("Accuracy: ",total2/total1)

In [10]:
def process_test_data():
    with open('dev-v1.1.json') as data_file:    
        data = json.load(data_file)
    category = questotag.keys()
    data2 = {}
    total1 = 0
    total2 = 0
    topics = data['data']
    with open("Question_answer_test","wb") as f:
        for t in topics: 
            true_answer_cnt=0
            total_cnt=0
            for cat in category:
                data2[cat] = get_paras(t, cat)
            for cat in category:
                for i in range(0,len(data2[cat])):
                    para = data2[cat][i][0]
                    question = data2[cat][i][1][0]
                    sent_indices = get_ans_sentences(para, question)
                    sent_indices
                    predicted_sentence = print_possible_sentence(para,sent_indices, cat)
                    answer =  data2[cat][i][2][0]
                    total_cnt+=1
                    total1+=1
                    cnd=0
                    if(predicted_sentence!=None):
                        for i in predicted_sentence:
                            if answer in i:
                                true_answer_cnt+=1
                                total2+=1
                                pickle.dump((question+" "+i,1),f)
                                cnd=1
                                break
                            else:
                                pickle.dump((question+" "+i,0),f)
                    if(cnd==0):
                        pickle.dump((question+" "+answer,1),f)
    print("number of answers matched with possible answers",total2,"from total of",total1)
    print("Accuracy: ",total2/total1)

In [11]:
#Run only if you want to change the "Question_answer" and  "Question_answer_test" file
process_train_data()
process_test_data()

number of answers matched with possible answers 2099 from total of 3395
Accuracy:  0.6182621502209131
number of answers matched with possible answers 2099 from total of 3395
Accuracy:  0.6182621502209131


In [9]:
def create_train_test():
    objs=[]
    with open("Question_answer","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X = []
    Y = []
    
    for o in objs:
        X.append(o[0])
        Y.append(o[1])
    vec = TfidfVectorizer()
    X_csr = vec.fit_transform(X)
    objs=[]
    with open("Question_answer_test_2","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X_val=X_csr[-100:,:]
    Y_val=Y[-100:]
    X_csr=X_csr[:-100,:]
    Y=Y[:-100]
    X_test = []
    Y_test= []
    for o in objs:
        X_test.append(o[0])
        Y_test.append(o[1])
    X_test_csr = vec.transform(X_test)
    return X_csr,Y,X_val,Y_val,X_test_csr,Y_test,X

In [27]:
def tunning_parameters(X,Y,X_test,y_test):
    tuned_parameters = [{'random_state': [2,42,None], 
                         'penalty': ['l1', 'l2'],
                         'C': [1,0.1],
                         'class_weight': ['balanced',None]}]
    scores = ['precision', 'recall','f1']
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()
        clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5,
                           scoring='%s_macro' % score)
        clf.fit(X,y)
        print("Best parameters set found on validation set:")
        print()
        print(clf.best_params_)
        print("Grid scores on validation set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is evaluated on the full validation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(sk.metrics.classification_report(y_true, y_pred))
    return clf.best_params_

In [29]:
X, y, X_val, Y_val, X_test, y_test = create_train_test()
result=tunning_parameters(X,y,X_val,Y_val)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 0.1, 'random_state': 2, 'penalty': 'l1', 'class_weight': None}
Grid scores on development set:

0.616 (+/-0.032) for {'C': 1, 'random_state': 2, 'penalty': 'l1', 'class_weight': 'balanced'}
0.616 (+/-0.032) for {'C': 1, 'random_state': 42, 'penalty': 'l1', 'class_weight': 'balanced'}
0.616 (+/-0.032) for {'C': 1, 'random_state': None, 'penalty': 'l1', 'class_weight': 'balanced'}
0.623 (+/-0.030) for {'C': 1, 'random_state': 2, 'penalty': 'l2', 'class_weight': 'balanced'}
0.623 (+/-0.030) for {'C': 1, 'random_state': 42, 'penalty': 'l2', 'class_weight': 'balanced'}
0.623 (+/-0.030) for {'C': 1, 'random_state': None, 'penalty': 'l2', 'class_weight': 'balanced'}
0.655 (+/-0.036) for {'C': 1, 'random_state': 2, 'penalty': 'l1', 'class_weight': None}
0.655 (+/-0.036) for {'C': 1, 'random_state': 42, 'penalty': 'l1', 'class_weight': None}
0.655 (+/-0.036) for {'C': 1, 'random_state': None, 'penalty'

In [32]:
print(result)

{'C': 0.1, 'random_state': 2, 'penalty': 'l1', 'class_weight': None}


In [10]:
#Logistic Regression
def logistic_model(X_csr,Y,X_val,Y_val,X_test_csr,Y_test,X): 
    print('Training data shape: %s\n' % str(X_csr.shape))
    clf = LogisticRegression(C= 0.1, random_state=2, penalty='l1', class_weight=None)
    clf.fit(X_csr, Y)
    acc=cross_val_score(clf,X_csr,Y,cv=10)
    print("Accuracy on training dataset",np.mean(acc))
    print('Test data shape: %s\n' % str(X_test_csr.shape))
    y_pred=clf.predict(X_test_csr)
    print(sk.metrics.classification_report(Y_test,y_pred))
    print ("\nPrecision", sk.metrics.precision_score(Y_test, y_pred))
    print ("\nRecall", sk.metrics.recall_score(Y_test, y_pred))
    print ("\nf1_score", sk.metrics.f1_score(Y_test, y_pred))
    print ("\nconfusion_matrix")
    print (sk.metrics.confusion_matrix(Y_test, y_pred))
    question_types_count={'what':0, 'why':0, 'when':0,'where':0, 'how':0, 'which':0,'who':0, 'whom':0}
    question_words = ['what', 'why', 'when','where', 'how', 'which','who', 'whom']
    word_count = Counter()
    print(Counter(y_pred))
    for i,y in enumerate(y_pred):
        if y == 1:        
            all_qs = X[i].lower()
            for q in all_qs.split():
                if q in question_types_count.keys():
                    question_types_count[q] += 1
    print(question_types_count)
    word_count.clear()
    return 

In [12]:
logistic_model(X, y, X_val, Y_val, X_test, y_test, X_main)

Training data shape: (87707, 54867)

Accuracy on training dataset 0.754113348708
Test data shape: (10368, 54867)

             precision    recall  f1-score   support

          0       0.76      0.94      0.84      6973
          1       0.74      0.39      0.51      3395

avg / total       0.75      0.76      0.73     10368


Precision 0.744753261486

Recall 0.386745213549

f1_score 0.509112058938

confusion_matrix
[[6523  450]
 [2082 1313]]
Counter({0: 8605, 1: 1763})
{'where': 193, 'who': 446, 'how': 9, 'which': 159, 'what': 1123, 'why': 13, 'whom': 15, 'when': 225}


In [38]:
#MLP Classifier
def neural_model():
    objs=[]
    with open("Question_answer","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X = []
    Y= []
    for o in objs:
        X.append(o[0])
        Y.append(o[1])
    vec = TfidfVectorizer()
    X_csr = vec.fit_transform(X)
    print('Training data shape: %s\n' % str(X_csr.shape))
    clf = MLPClassifier(random_state=1,solver="lbfgs", hidden_layer_sizes=(5,2))
    clf.fit(X_csr, Y)
    acc=cross_val_score(clf,X_csr,Y,cv=10)
    print("Accuracy on training dataset",np.mean(acc))
    objs=[]
    with open("Question_answer_test_2","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X_test = []
    Y_test= []
    
    for o in objs:
        X_test.append(o[0])
        Y_test.append(o[1])
    X_test_csr = vec.transform(X_test)
    print('Test data shape: %s\n' % str(X_test_csr.shape))
    y_pred=clf.predict(X_test_csr)
    print(sk.metrics.classification_report(Y_test,y_pred))
    print ("\nPrecision", sk.metrics.precision_score(Y_test, y_pred))
    print ("\nRecall", sk.metrics.recall_score(Y_test, y_pred))
    print ("\nf1_score", sk.metrics.f1_score(Y_test, y_pred))
    print ("\nconfusion_matrix")
    print (sk.metrics.confusion_matrix(Y_test, y_pred))

In [39]:
neural_model()

Training data shape: (87807, 54867)

Accuracy on training dataset 0.562632914719
Test data shape: (10368, 54867)

             precision    recall  f1-score   support

          0       0.79      0.66      0.72      6973
          1       0.48      0.64      0.55      3395

avg / total       0.69      0.65      0.66     10368


Precision 0.479079034758

Recall 0.637407952872

f1_score 0.54701718908

confusion_matrix
[[4620 2353]
 [1231 2164]]


In [4]:
##########################################
####### Keras sequential method ##########
##########################################
def keras_model():
    objs=[]
    with open("Question_answer","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X = []
    Y= []
    for o in objs:
        X.append(o[0])
        Y.append(o[1])
    vec = TfidfVectorizer()
    X_csr = vec.fit_transform(X)
    X_csr=X_csr.toarray()
    print('Training data shape: %s\n' % str(X_csr.shape))
   
    Y = to_categorical(Y)
    
    
    model=Sequential()
    model.add(Dense(20,input_dim=X_csr.shape[1]))
    model.add(Activation('relu'))
    print(model.output_shape)
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

    
    model.fit(X_csr,Y,epochs=2)
    objs=[]
    score = model.evaluate(X_csr, Y)
    y_true=Y_test
    print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    
    
    with open("Question_answer_test_2","rb") as f:
        while 1:
            try:
                objs.append(pickle.load(f))
            except EOFError:
                break
    X_test = []
    Y_test= []
    
    for o in objs:
        X_test.append(o[0])
        Y_test.append(o[1])
    X_test_csr = vec.transform(X_test)
    X_test_csr=X_test_csr.toarray()
    Y_test = to_categorical(Y_test)
    print('Test data shape: %s\n' % str(X_test_csr.shape))
    
    loss_metrics=model.evaluate(X_test_csr,Y_test)
    classes=model.predict(X_test_csr)
    score = model.evaluate(X_test_csr, Y_test)
    y_true=Y_test
    print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    y_pred=[np.argmax(c) for c in classes]
    y_pred = to_categorical(y_pred)
    print(Counter([np.argmax(c) for c in classes]))
    print(sk.metrics.classification_report(y_true,y_pred))
    print ("\nPrecision", sk.metrics.precision_score(y_true, y_pred))
    print ("\nRecall", sk.metrics.recall_score(y_true, y_pred))
    print ("\nf1_score", sk.metrics.f1_score(y_true, y_pred))
    print ("\nconfusion_matrix")
    print (sk.metrics.confusion_matrix(y_true, y_pred))

In [5]:
keras_model()

Training data shape: (87807, 54867)

(None, 20)
Epoch 1/2
87807/87807 [==============================] - 125s - loss: 0.5160 - acc: 0.7520   
Epoch 2/2
87807/87807 [==============================] - 129s - loss: 0.4534 - acc: 0.7929   
Test data shape: (10368, 54867)

10336/10368 [============================>.] - ETA: 0s
acc: 72.99%
Counter({0: 7196, 1: 3172})
             precision    recall  f1-score   support

          0       0.79      0.82      0.80      6973
          1       0.59      0.55      0.57      3395

avg / total       0.73      0.73      0.73     10368



ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.